# Andmete puhastamine

Siin töötleme varasemas etapis kogutud toorandmeid (`course_details_merged.csv`).
Faili leiad projekti ÜHISEST repositooriumist. 

**Sisend:** `../data/toorandmed_aasta.csv`

**Väljund:** `../data/andmed_aasta.csv`

### Samm 0: Teekide laadimine ja failiteede seadistamine 
Laeme vajalikud Pythoni teegid ja fikseerime sisend- ja väljundfailide asukohad. Loeme sisse puhastamata andmed.

In [6]:
import pandas as pd
import json
import numpy as np
import os

# Konfigureerime Pandase sätteid loetavuse huvides
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

# Sisend- ja väljundfailid
INPUT_FILE = 'Data/toorandmed_aasta.csv'
OUTPUT_FILE = 'Data/andmed_aasta.csv'


if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE, low_memory=False)
    print(f"Toorandmed loetud. Andmestiku suurus: {df.shape[0]} rida, {df.shape[1]} veergu.")
else:
    # Kui faili pole, loome testimiseks tühja DataFrame'i või viskame vea
    raise FileNotFoundError(f"Viga: Sisendfaili '{INPUT_FILE}' ei leitud! Kontrolli failiteed.")

# Kiire pilk andmestruktuurile
df.head(2)

print("Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.")

Toorandmed loetud. Andmestiku suurus: 3156 rida, 229 veergu.
Samm 1: Teegid laetud ja seadistused tehtud. Andmed sisse loetud.


### Samm 1: eemaldame andmed, mida me ei taha

Siin võib filtreerida välja ridu mõne tunnuse alusel kui teame, et oma rakenduses vajame ainult teatud tüüpi andmeid, hetkel me seda ei teinud, aga kui tahaksime nt ainult kevade aineid, saaksime seda siin teha.

In [7]:
print("Samm 1: Rakendan eelfiltreid...")

before = len(df)

# 1) eemaldada ained, mille kestus on rohkem kui 1 semester
# (kolumn: additional_info__duration_in_semesters)
dur_col = 'additional_info__duration_in_semesters'
if dur_col in df.columns:
    df[dur_col] = pd.to_numeric(df[dur_col], errors='coerce')
    df = df[(df[dur_col].isna()) | (df[dur_col] <= 1)]
else:
    print(f"HOIATUS: veerg '{dur_col}' puudub, kestuse filtrit ei rakendatud.")

# 2) eemaldada kaitsmise ained
# Heuristika: kood/pealkiri sisaldab "kaitsm", "defence", "thesis defence" jne
title_cols = [c for c in ['title__et','version__title__et','title__en','version__title__en','code'] if c in df.columns]
if title_cols:
    mask_kaitsmine = False
    for c in title_cols:
        mask_kaitsmine = mask_kaitsmine | df[c].astype(str).str.lower().str.contains(r'kaitsm|defen|thesis\s*defen', regex=True, na=False)
    df = df[~mask_kaitsmine]
else:
    print("HOIATUS: ei leidnud title/code veerge, kaitsmise filtrit ei rakendatud.")

after = len(df)
print(f"Eelfiltrid tehtud: {before} -> {after} rida (eemaldati {before-after}).")


Samm 1: Rakendan eelfiltreid...
Eelfiltrid tehtud: 3156 -> 3155 rida (eemaldati 1).


### Samm 2: Puhastatud veergude lisamine

Valime välja huvipakkuvad veerud ning puhastame need. Antud sammu juures seisneb puhastamine õige veeru valimises - juhul kui huvipakkuv sisu võib olla kahes erinevas veerus, eelistame aine konkreetse versiooni infot.


Täpsem kirjeldus aine üldine info vs versiooni info probleemist:
* **Probleem:** Meil on topelt veerge eeldatavasti sama infoga (üldine aine info vs aine konkreetse versiooni info). Nt eestikeelne pealkiri võiks peituda nii `title__et` kui ka `version__title__et` veerus.
* **Lahendus:** Eelistame alati `version__` prefiksiga veerge, sest need kajastavad tegelikku aineprogrammi. Kui see puudub, võtame üldise info.

In [8]:
def resolve_fields(df):
    """
    Loob puhtad veerud, eelistades versiooni-põhist infot (version__*).
    Lisab ka hard-filter veerud ja kirjeldusse mineva sisu.
    """

    merge_mapping = [
        ('nimi_et', 'title__et', 'version__title__et'),
        ('nimi_en', 'title__en', 'version__title__en'),
        ('eap', 'credits', 'version__credits'),

        # kirjeldus ET (eelistame versiooni)
        ('kirjeldus_et', 'overview__description__et', 'version__overview__description__et'),

        # õpiväljundid (sul on nii tekstiväljad kui struktuursed — võtame tekstivälja kui olemas)
        ('opivaljundid_et', 'overview__learning_outcomes', 'version__overview__learning_outcomes'),
        ('opivaljundid_en', 'overview__learning_outcomes_text_en', 'version__overview__learning_outcomes_text_en'),

        # eeldusained
        ('eeldusained', 'additional_info__prerequisites', 'version__additional_info__prerequisites'),
    ]

    rename_mapping = [
        ('aine_kood', 'code'),

        # hard-filter: semester (kevad/sügis)
        ('semester', 'version__target__semester__et'),

        # hard-filter: eristav/mitteeristav (hindamisskaala)
        ('hindamisskaala_kood', 'version__grading__assessment_scale__code'),
        ('hindamisskaala_et', 'version__grading__assessment_scale__et'),

        # hard-filter: õppeaste (study_type)
        ('oppeaste_kood', 'version__target__study_type__code'),
        ('oppeaste_et', 'version__target__study_type__et'),

        # hard-filter: keel (target language)
        ('oppekeel_kood', 'version__target__language__code'),
        ('oppekeel_et', 'version__target__language__et'),

        # (valikuline) asukoht: linn (üks lihtne proxy)
        ('asukoht_linn', 'version__target__faculty__city'),

        # kestus
        ('kestus_sem', 'additional_info__duration_in_semesters'),

        # JSON väljad
        ('oppejoud_json', 'version__participants__lecturers'),
        ('toimumisajad_json', 'version__schedule__entries'),

        # hindamise tekstid (eksami tuvastamine kui üldse)
        ('hindamine_pre_et', 'version__grading__grade_preconditions__et'),
        ('hindamine_eval_et', 'version__grading__grade_evaluation__et'),
    ]

    for new_col, base, version in merge_mapping:
        base_exists = base in df.columns if base else False
        ver_exists = version in df.columns if version else False

        if base_exists and ver_exists:
            df[new_col] = df[version].fillna(df[base])
        elif ver_exists:
            df[new_col] = df[version]
        elif base_exists:
            df[new_col] = df[base]
        else:
            df[new_col] = np.nan

    for new_col, source in rename_mapping:
        df[new_col] = df[source] if source in df.columns else np.nan

    return df


df_resolved = resolve_fields(df.copy())
print("Samm 2: Puhastatud veerud + hard-filter veerud lisatud.")


Samm 2: Puhastatud veerud + hard-filter veerud lisatud.



### Samm 3: JSON väljade parsimine

See koodiplokk tegeleb tehniliste JSON-formaadis veergude teisendamisega inimloetavaks tekstiks. Eesmärk on eraldada keerulisest struktuurist vaid oluline sisu (nt nimed või nädalapäevad) ja vormistada see komadega eraldatud loeteluks.

Näide teisendamisest (õppejõud):
* Enne (Toores JSON): ``'[{"person_name": "Mari Mets", "id": 101}, {"person_name": "Jaan Kask", "id": 102}]'``
* Pärast (Puhastatud): ``"Jaan Kask, Mari Mets"``

In [11]:
import re
from datetime import time

# ---------- Robust JSON parse ----------
def parse_json_safe(x):
    """
    Accepts:
      - NaN/None -> None
      - already-parsed list/dict -> returned as-is
      - JSON string -> parsed
    Returns list/dict or None.
    """
    if x is None or (isinstance(x, float) and pd.isna(x)) or (isinstance(x, str) and x.strip() == ""):
        return None
    if isinstance(x, (list, dict)):
        return x
    if isinstance(x, str):
        s = x.strip()
        try:
            return json.loads(s)
        except Exception:
            return None
    return None


# ---------- Lecturers extraction ----------
LECTURER_KEYS = ["person_name", "name", "full_name", "display_name", "teacher_name", "lecturer_name"]

def extract_lecturers(x):
    data = parse_json_safe(x)
    if not data:
        return None

    # data can be list or dict
    items = data if isinstance(data, list) else [data]

    names = []
    for it in items:
        if not isinstance(it, dict):
            continue

        # try known keys first
        for k in LECTURER_KEYS:
            v = it.get(k)
            if isinstance(v, str) and v.strip():
                names.append(v.strip())
                break

        # sometimes it's nested (e.g., {"person": {"name": "..."}}
        if not any(isinstance(it.get(k), str) and it.get(k).strip() for k in LECTURER_KEYS):
            for v in it.values():
                if isinstance(v, dict):
                    for k in LECTURER_KEYS:
                        vv = v.get(k)
                        if isinstance(vv, str) and vv.strip():
                            names.append(vv.strip())
                            break

    names = sorted(set(names))
    return ", ".join(names) if names else None


# ---------- Schedule extraction ----------
ET_DAYS = ["Esmaspäev", "Teisipäev", "Kolmapäev", "Neljapäev", "Reede", "Laupäev", "Pühapäev"]
EN_DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

EN_TO_ET = dict(zip(EN_DAYS, ET_DAYS))
ET_TO_ORDER = {d: i for i, d in enumerate(ET_DAYS)}

# detect day strings anywhere
DAY_PATTERN = re.compile(
    r"\b(" + "|".join(EN_DAYS + ET_DAYS + [d.lower() for d in ET_DAYS]) + r")\b",
    flags=re.IGNORECASE
)

def _normalize_day(day):
    if not day:
        return None
    d = str(day).strip()
    # english -> estonian
    if d in EN_TO_ET:
        return EN_TO_ET[d]
    # title-case estonian
    dl = d.lower()
    for et in ET_DAYS:
        if dl == et.lower():
            return et
    return None

def _parse_time_str(t):
    """
    Accepts "HH:MM", "HH:MM:SS", sometimes "HH.MM".
    Returns 'HH:MM' string or None.
    """
    if t is None or (isinstance(t, float) and pd.isna(t)):
        return None
    s = str(t).strip()
    if not s:
        return None
    s = s.replace(".", ":")
    # keep first 5 of HH:MM if it matches
    m = re.match(r"^(\d{1,2}):(\d{2})", s)
    if not m:
        return None
    hh = int(m.group(1))
    mm = int(m.group(2))
    if 0 <= hh <= 23 and 0 <= mm <= 59:
        return f"{hh:02d}:{mm:02d}"
    return None

def extract_schedule_info(x):
    """
    Tries to extract:
      - days (Estonian)
      - time ranges if any
    Returns: (days_str, times_str, combined_str)
    """
    data = parse_json_safe(x)
    if not data:
        return (None, None, None)

    entries = data if isinstance(data, list) else [data]

    days = set()
    times = set()

    for e in entries:
        if not isinstance(e, dict):
            # maybe raw string
            if isinstance(e, str):
                for m in DAY_PATTERN.findall(e):
                    nd = _normalize_day(m)
                    if nd: days.add(nd)
            continue

        # 1) try explicit weekday fields
        for key in ["weekday", "day", "day_of_week", "week_day", "weekday_name", "weekday_en", "weekday_et"]:
            if key in e:
                nd = _normalize_day(e.get(key))
                if nd:
                    days.add(nd)

        # 2) try timestamp fields -> day-of-week
        for key in ["time", "start_time", "start", "date", "datetime", "begin"]:
            val = e.get(key)
            if val:
                dt = pd.to_datetime(val, errors="coerce")
                if not pd.isna(dt):
                    days.add(EN_TO_ET.get(dt.day_name(), dt.day_name()))

        # 3) scan any text fields for weekday names
        for v in e.values():
            if isinstance(v, str):
                for m in DAY_PATTERN.findall(v):
                    nd = _normalize_day(m)
                    if nd: days.add(nd)

        # 4) time range extraction (common key variants)
        start_t = _parse_time_str(e.get("start_time") or e.get("start") or e.get("from") or e.get("begin_time"))
        end_t   = _parse_time_str(e.get("end_time")   or e.get("end")   or e.get("to")   or e.get("finish_time"))

        if start_t and end_t:
            times.add(f"{start_t}-{end_t}")
        elif start_t:
            times.add(start_t)

    days_sorted = sorted(days, key=lambda d: ET_TO_ORDER.get(d, 99))
    times_sorted = sorted(times)

    days_str = ", ".join(days_sorted) if days_sorted else None
    times_str = ", ".join(times_sorted) if times_sorted else None

    combined_parts = []
    if days_str: combined_parts.append(days_str)
    if times_str: combined_parts.append(times_str)
    combined_str = " | ".join(combined_parts) if combined_parts else None

    return (days_str, times_str, combined_str)


# ---------- Apply to dataframe ----------
print("Ekstraheerime JSON väljadest infot (parendatud)...")

df_resolved['oppejoud'] = df_resolved['oppejoud_json'].apply(extract_lecturers)

sched = df_resolved['toimumisajad_json'].apply(extract_schedule_info)
df_resolved['toimumispaevad'] = sched.apply(lambda t: t[0])
df_resolved['toimumisajad_kell'] = sched.apply(lambda t: t[1])
df_resolved['toimumisinfo'] = sched.apply(lambda t: t[2])

print("Samm 3: JSON töötlemine valmis (parendatud).")
print(df_resolved[['aine_kood', 'oppejoud', 'toimumispaevad', 'toimumisajad_kell', 'toimumisinfo']].head(5))


Ekstraheerime JSON väljadest infot (parendatud)...
Samm 3: JSON töötlemine valmis (parendatud).
     aine_kood  \
0  OIEO.06.046   
1  KKSB.05.092   
2  ARKI.02.030   
3  LOFY.04.041   
4  ARSK.01.030   

                                                                             oppejoud  \
0                                                                           Gea Lepik   
1                                                                    Kadri Medijainen   
2  Jekaterina Nerman, Jürgen Rünk, Pille-Riin Värk, Ragnar Lõivukene, Viljo Kübarsepp   
3                                                                       Stefan Groote   
4                  Ingrid Reppo, Olga Gusseva, Piia Muru, Sille Kaseleht, Vallo Volke   

       toimumispaevad toimumisajad_kell        toimumisinfo  
0                None              None                None  
1  Neljapäev, Laupäev              None  Neljapäev, Laupäev  
2                None              None                None  
3           

### Samm 4 (Valikuline): Info koondamine mitmest allikast

Kui vajalik info on mitmes veerus, saame need ühendada. Hetkel teeme seda näiteks hindamisinfoga.

In [12]:
grading_source_cols = [
    ('Miinimumnõuded', 'hindamine_pre_et'),
    ('Hindamismeetod', 'hindamine_eval_et')
]

def combine_grading_info(row):
    parts = []
    for label, col in grading_source_cols:
        val = row.get(col)
        if pd.notna(val) and str(val).strip():
            parts.append(f"{label}: {str(val).strip()}")
    return " | ".join(parts) if parts else None

df_resolved['hindamine_info'] = df_resolved.apply(combine_grading_info, axis=1)

# Nišš: tuvastame kas tekstides esineb "eksam"
def detect_exam(text):
    if pd.isna(text): 
        return False
    t = str(text).lower()
    return ("eksam" in t) or ("exam" in t)

df_resolved['kas_eksam'] = df_resolved['hindamine_info'].apply(detect_exam)

print("Samm 4: Hindamisinfo koondatud + kas_eksam tuletatud.")


Samm 4: Hindamisinfo koondatud + kas_eksam tuletatud.


### Hard-filter veerg: “eristav/mitteeristav” (lihtne normaliseerimine)


In [13]:
def normalize_eristav(scale_et):
    if pd.isna(scale_et):
        return None
    s = str(scale_et).lower()
    # tüüpilised: "Eristav hindamine", "Mitteeristav hindamine"
    if "mitte" in s and "erist" in s:
        return "mitteeristav"
    if "erist" in s:
        return "eristav"
    return None

df_resolved['hindamine_tuup'] = df_resolved['hindamisskaala_et'].apply(normalize_eristav)



### Samm 5: Lõplik viimistlus

Valime lõplikud veerud. 

In [14]:
# Koondtekst RAG jaoks: kirjeldus + eeldusained + õpiväljundid + lisainfo
def build_rag_text(row):
    parts = []

    # Põhiinfo
    if pd.notna(row.get('nimi_et')):
        parts.append(f"Nimi: {row.get('nimi_et')}")
    if pd.notna(row.get('aine_kood')):
        parts.append(f"Kood: {row.get('aine_kood')}")
    if pd.notna(row.get('eap')):
        parts.append(f"EAP: {row.get('eap')}")

    # Kirjeldusse lisatav info
    if pd.notna(row.get('kirjeldus_et')):
        parts.append(f"Kirjeldus: {row.get('kirjeldus_et')}")
    if pd.notna(row.get('eeldusained')):
        parts.append(f"Eeldusained: {row.get('eeldusained')}")
    # õpiväljundid: kui ET on tühi, kasuta EN
    lo = row.get('opivaljundid_et')
    if pd.isna(lo) or str(lo).strip()=="":
        lo = row.get('opivaljundid_en')
    if pd.notna(lo):
        parts.append(f"Õpiväljundid: {lo}")

    # soovi korral lisa hindamine
    if pd.notna(row.get('hindamine_info')):
        parts.append(f"Hindamine: {row.get('hindamine_info')}")

    return "\n".join(parts)

df_resolved['kirjeldus'] = df_resolved.apply(build_rag_text, axis=1)

final_cols = [
    # ident
    'aine_kood', 'nimi_et', 'nimi_en', 'eap',

    # hard-filterid
    'semester', 'hindamine_tuup', 'oppeaste_et', 'oppekeel_et', 'asukoht_linn', 'kas_eksam',

    # kirjeldusse minev sisu eraldi + koondtekst
    'kirjeldus_et', 'eeldusained', 'opivaljundid_et', 'opivaljundid_en',
    'hindamine_info', 'toimumisajad', 'oppejoud',
    'kirjeldus'
]

existing_cols = [c for c in final_cols if c in df_resolved.columns]
df_final = df_resolved[existing_cols].copy()

print(f"Samm 5: Lõplik ridade arv: {len(df_final)}, veerge: {len(df_final.columns)}")
df_final.head(2)


Samm 5: Lõplik ridade arv: 3155, veerge: 18


,aine_kood,nimi_et,nimi_en,eap,semester,hindamine_tuup,oppeaste_et,oppekeel_et,asukoht_linn,kas_eksam,kirjeldus_et,eeldusained,opivaljundid_et,opivaljundid_en,hindamine_info,toimumisajad,oppejoud,kirjeldus
0,OIEO.06.046,Rahvusvaheline eraõigus,Private International Law,6.0,kevad,eristav,päevaõpe,eesti keel,Tartu linn,True,"Kursuse raames käsitletakse rahvusvahelise eraõiguse põhiteemasid, terminoloogiat ja kujunemist,...",NaN,"[{""en"": ""At the end of the course the student is able to:"", ""et"": ""Kursuse lõpuks peab üliõpilan...",- At the end of the course the student is able to:\n- -\texplain the nature of the main institut...,Miinimumnõuded: Aktiivne osalemine seminarides. | Hindamismeetod: 100% kirjalik eksam.,None,Gea Lepik,Nimi: Rahvusvaheline eraõigus\nKood: OIEO.06.046\nEAP: 6.0\nKirjeldus: Kursuse raames käsitletak...
1,KKSB.05.092,Praktiseerimine töökeskkonnas,Practice in the Work Environment,3.0,kevad,eristav,päevaõpe,eesti keel,Tartu linn,False,Aines Praktiseerimine töökeskkonnas laiendab üliõpilane oma silmaringi füsioteraapia valdkonnas ...,NaN,"[{""en"": ""Student:"", ""et"": ""Õpilane:""}, {""en"": ""understands and follows the work ethic and organi...",- Student:\n- understands and follows the work ethic and organization in the work environment;;\...,Miinimumnõuded: Nõutava tähtaegne esitamine,"Neljapäev, Laupäev",Kadri Medijainen,Nimi: Praktiseerimine töökeskkonnas\nKood: KKSB.05.092\nEAP: 3.0\nKirjeldus: Aines Praktiseerimi...


### Samm 6: Salvestamine ja kontroll

Kontrollime mõningaid kvaliteediaspekte ning salvestame puhastatud faili.

* Salvesta puhastatud fail.
* Arvuta puuduvate väärtuste hulk igas veerus.
* Leia kategooriliste tunnustega veergude [keel, semester, õppeaste, …] enim levinud väärtused (näiteks df[tunnus].count().head(5))
* Näita "Sissejuhatus andmeteadusesse” aine kogu puhastatud kujul väljund lihtsasti loetaval kujul (võid küsida LLMilt abi, et väljund hästi kenasti vormistada, et see ei oleks lihtsalt välja prinditud andmerida).
* Lisada tunnus kirjeldus, sisaldab ühe aine koguinfot (kõik veerud ühendatud suureks tekstiks. Teha tunnuse kirjeldus tähemärkide arvu statistika (describe() funktsioon võib aidata, aga võib kasutada ka visualiseerimist), et teaksime, kui palju infot me RAGile ette anda kavatseme

In [15]:
# 1) Salvesta
df_final.to_csv(OUTPUT_FILE, index=False)
print(f"Samm 6: Andmestik salvestatud: {OUTPUT_FILE}")

# 2) Puuduvate väärtuste hulk igas veerus
print("\nPuuduvate väärtuste hulk igas veerus:")
missing_per_col = df_final.isna().sum().sort_values(ascending=False)
display(missing_per_col)

# 3) Kategooriliste tunnustega veergude enim levinud väärtused
cat_cols = [c for c in ['oppekeel_et','semester','oppeaste_et','hindamine_tuup','asukoht_linn'] if c in df_final.columns]
print("\nKategooriliste tunnuste TOP väärtused:")
for c in cat_cols:
    print(f"\n== {c} ==")
    display(df_final[c].value_counts(dropna=False).head(5))

# 4) Näita “Sissejuhatus andmeteadusesse” aine puhastatud kujul (ilusalt)
# Proovime leida kas pealkirjast või koodist
query = "Sissejuhatus andmeteadusesse"
mask = False
if 'nimi_et' in df_final.columns:
    mask = mask | df_final['nimi_et'].astype(str).str.contains(query, case=False, na=False)
if 'aine_kood' in df_final.columns:
    mask = mask | df_final['aine_kood'].astype(str).str.contains("andmetead", case=False, na=False)

matches = df_final[mask]
print(f"\nLeitud vasteid '{query}': {len(matches)}")
if len(matches) > 0:
    row = matches.iloc[0]
    print("\n--- Aine puhastatud kujul (loetav) ---")
    for col, val in row.items():
        print(f"{col}: {val}")
else:
    print("Ei leidnud seda ainet nime/koodi järgi. Kontrolli, kuidas see andmestikus täpselt kirjas on.")

# 5) Kirjeldus tähemärkide statistika (RAG sisendi pikkus)
if 'kirjeldus' in df_final.columns:
    df_final['kirjeldus_pikkus'] = df_final['kirjeldus'].fillna("").astype(str).str.len()
    print("\nKirjeldus (koondtekst) tähemärkide statistika:")
    display(df_final['kirjeldus_pikkus'].describe())
else:
    print("HOIATUS: 'kirjeldus' veerg puudub, pikkuse statistikat ei saa teha.")



Samm 6: Andmestik salvestatud: Data/andmed_aasta.csv

Puuduvate väärtuste hulk igas veerus:


eeldusained        2590
toimumisajad       2011
hindamine_info      231
oppejoud             22
asukoht_linn         21
semester             21
hindamine_tuup       21
oppeaste_et          21
oppekeel_et          21
opivaljundid_et       1
opivaljundid_en       1
aine_kood             0
nimi_et               0
eap                   0
kas_eksam             0
nimi_en               0
kirjeldus_et          0
kirjeldus             0
dtype: int64


Kategooriliste tunnuste TOP väärtused:

== oppekeel_et ==


oppekeel_et
eesti keel      2326
inglise keel     808
NaN               21
Name: count, dtype: int64


== semester ==


...

hindamine_tuup
eristav    3134
None         21
Name: count, dtype: int64


== asukoht_linn ==


asukoht_linn
Tartu linn    3133
NaN             21
Tartu            1
Name: count, dtype: int64


Leitud vasteid 'Sissejuhatus andmeteadusesse': 1

--- Aine puhastatud kujul (loetav) ---
aine_kood: LTAT.02.002
nimi_et: Sissejuhatus andmeteadusesse
nimi_en: Introduction to Data Science
eap: 6.0
semester: sügis
hindamine_tuup: eristav
oppeaste_et: päevaõpe
oppekeel_et: inglise keel
asukoht_linn: Tartu linn
kas_eksam: False
kirjeldus_et: Kursus annab lühiülevaate andmeteaduse põhimõistetest, põhiprintsiipidest ning praktilistest aspektidest. Lõppeesmärk on õppida planeerima ning läbi viima lihtsat praktilist andmeteaduse projekti. Kursus käsitleb peamisi andmete kirjeldava analüüsi ning visualiseerimise meetodeid, sagedaste mustrite otsimist, klasteranalüüsi, peakomponentanalüüsi, peamisi masinõppemeetodeid klassifitseerimise ja regressiooni jaoks (sealhulgas sügavaid neurovõrke), andmete haldamist ning statistiliste testide interpreteerimist. Tutvutakse andmeteaduse projekti põhiliste etappidega ning tarkvaraliste töövahenditega. Peamiseks töövahendiks Õpime projekti ellu viima base

count     3155.000000
mean      2091.991759
std       1027.583743
min        116.000000
25%       1397.000000
50%       1887.000000
75%       2579.000000
max      10095.000000
Name: kirjeldus_pikkus, dtype: float64